In [21]:
import cv2

if hasattr(cv2, "aruco"):
    print("Aruco is available.")


Aruco is available.


In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture("aruco.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
frame_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter("aruco_green_overlay.mp4", fourcc, fps, (frame_w, frame_h))

dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
parameters = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(dictionary, parameters)

expected_ids = [0, 1, 2, 3]
margin = 10
marker_size = 80

last_screen_pts = None

def shift_corner(corner, dx, dy):
    return np.array([corner[0] + dx, corner[1] + dy])

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    overlay = frame.copy()
    corners, ids, _ = detector.detectMarkers(frame)

    screen_pts = None
    has_full_detection = False

    if ids is not None and len(ids) >= 4:
        detected = {int(id_): corners[i][0] for i, id_ in enumerate(ids.flatten()) if int(id_) in expected_ids}

        if all(k in detected for k in expected_ids):
            # Get corners of each marker
            tl = shift_corner(detected[0][0], -margin, -margin)  # top-left corner of ID 0
            tr = shift_corner(detected[1][1], margin, -margin)   # top-right corner of ID 1
            br = shift_corner(detected[2][2], margin, margin)    # bottom-right corner of ID 2
            bl = shift_corner(detected[3][3], -margin, margin)   # bottom-left corner of ID 3

            screen_pts = np.array([tl, tr, br, bl], dtype=np.float32)
            last_screen_pts = screen_pts.copy()
            has_full_detection = True
        else:
            screen_pts = last_screen_pts
    else:
        screen_pts = last_screen_pts

    if screen_pts is not None:
        screen_pts_int = screen_pts.astype(np.int32)
        green_overlay = np.zeros_like(frame)
        cv2.fillPoly(green_overlay, [screen_pts_int], (0, 255, 0))

        alpha = 0.3
        overlay = cv2.addWeighted(frame, 1 - alpha, green_overlay, alpha, 0)

        # Draw boundary only when detection was fresh (for debug)
        color = (0, 255, 0) if has_full_detection else (0, 180, 255)
        cv2.polylines(overlay, [screen_pts_int], True, color, 2)

    out.write(overlay)

cap.release()
out.release()
print("✅ Done — with margin only on valid frames, and fallback only when needed.")


✅ Done — with margin only on valid frames, and fallback only when needed.
